In [1]:
import pandas as pd
import numpy as np
from Data_Processing import DataProcessing
from sklearn.model_selection import train_test_split
import joblib

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend as K
from keras.callbacks import EarlyStopping
from datetime import datetime
from sklearn.metrics import mean_squared_error

In [2]:
pop = pd.read_csv('../Data/population.csv')
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [3]:
X, y, test = DataProcessing(train, test, pop)

In [4]:
X.shape

(478741, 78)

## Neural Network

In [48]:
mc = ModelCheckpoint(f'../Models/Neural_Network.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True)

model = keras.Sequential([
    #keras.layers.InputLayer(78),
    keras.layers.Dense(78, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(25, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    #keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    #keras.layers.BatchNormalization(),
    keras.layers.Dense(3, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='selu', kernel_initializer='lecun_normal')
])

In [67]:
model = keras.Sequential([
    keras.layers.Dense(78, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    #keras.layers.Dense(25, activation='relu'),
    #keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='relu')
])

In [68]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
             loss='mean_squared_error',
             metrics=[keras.metrics.RootMeanSquaredError()])

In [69]:
history = model.fit(
    X,
    y,
    batch_size=2000,
    epochs=5000,
    validation_split=0.2,
    callbacks=[mc, early_stopping],
    shuffle=True,
    use_multiprocessing=True
    )

Epoch 1/5000
174/192 [==========================>...] - ETA: 0s - loss: 1.0072 - root_mean_squared_error: 1.0036
Epoch 1: val_loss did not improve from 0.65557
192/192 [==============================] - 1s 3ms/step - loss: 0.9814 - root_mean_squared_error: 0.9907 - val_loss: 0.7131 - val_root_mean_squared_error: 0.8444
Epoch 2/5000
170/192 [=========================>....] - ETA: 0s - loss: 0.7072 - root_mean_squared_error: 0.8410
Epoch 2: val_loss did not improve from 0.65557
192/192 [==============================] - 0s 3ms/step - loss: 0.7073 - root_mean_squared_error: 0.8410 - val_loss: 0.6930 - val_root_mean_squared_error: 0.8325
Epoch 3/5000
191/192 [============================>.] - ETA: 0s - loss: 0.6937 - root_mean_squared_error: 0.8329
Epoch 3: val_loss did not improve from 0.65557
192/192 [==============================] - 0s 3ms/step - loss: 0.6937 - root_mean_squared_error: 0.8329 - val_loss: 0.6834 - val_root_mean_squared_error: 0.8267
Epoch 4/5000
181/192 [===============

# Validation

In [70]:
model = load_model('../Models/Neural_Network.h5')

y_true = y
y_pred = model.predict(X)

from sklearn.metrics import r2_score

r2_score(y_true, y_pred)

0.012644682649172734

In [71]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')

In [72]:
train_zips = train_df['postcode']

In [73]:
df = pd.DataFrame({'postcode': train_zips, 'y_true': y_true.flatten(), 'y_pred': y_pred.flatten()}, columns=['postcode', 'y_true', 'y_pred'])
df = df.groupby('postcode').mean()

In [74]:
df

,y_true,y_pred
postcode,,
AB10 1AU,1.500000,1.415062
AB10 1BD,1.200000,1.425135
AB10 1BW,2.000000,1.410933
AB10 1HT,1.000000,1.497033
AB10 1NJ,1.428571,1.445176
...,...,...
ZE2 9QJ,1.500000,1.445190
ZE2 9QS,1.600000,1.484921
ZE2 9RE,1.500000,1.359639


# Submission Set

In [75]:
y_pred = model.predict(test)

test_zips = test_df['postcode']
test_df = pd.DataFrame({'postcode': test_zips, 'Accident_risk_index': y_pred.flatten()}, columns=['postcode', 'Accident_risk_index'])

submission = test_df.groupby('postcode').mean().reset_index()

submission.to_csv('../Submissions/First_Try.csv', index=False)
submission

,postcode,Accident_risk_index
0,AB10 1AU,1.395012
1,AB10 1PG,1.365194
2,AB10 1TT,1.546399
3,AB10 1YP,1.417821
4,AB10 6LQ,1.474229
...,...,...
49767,ZE2 9LZ,1.477347
49768,ZE2 9RE,1.386834
49769,ZE2 9RJ,1.355922
49770,ZE2 9SB,1.487988
